<a href="https://colab.research.google.com/github/kameda-yoshinari/DataAlgo-UT/blob/main/DataAlgo_UT(014)_KnapSack_BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.3. 分岐限定法

バックトラック法では，解の条件を利用して，解候補の数を絞り込んでいた．

これをさらに進めて，解候補が解の条件を満たすと見込めなくなった時点で解の探索を打ち切ることを考える．

これを分岐限定法という．

本節では，ナップサック問題 (Knapsack problem) という有名な問題を取り上げて，全解探索法，バックトラック法，分岐限定法の三つを学習する．

**いつもの約束**  
１つのコードセルだけの実行は Ctrl + Enter．  
エディタで「インデント幅（スペース）は4で表示」「行番号を表示」「インデントガイドを表示」．  
内部では日本語はUTF-8で表現されている．


# 準備

インスタンスに接続し起動する．  
下記の手順でGoogle Driveをマウントする．  
マウント先に移動し，作業フォルダとする．  
これによって，インスタンスがリセットされてもGoogle Drive内にファイルが保存されるようにする．

In [ ]:
!echo "Google Driveをマウントします"
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
!echo "今回の作業用フォルダを作成しそこに移動します"
%cd /content/drive/My\ Drive/
%mkdir -p 202004_DataAlgo/DA2021_014
%cd       202004_DataAlgo/DA2021_014
!ls
!echo "日本時間表示"
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Japan /etc/localtime
!date

# ナップサック問題 (Knapsack problem)

**問題**

今，美術館にN個の展示がある．もし貴方が稀代の大泥棒ルパン三世だったとしよう．ある夜，背中に W kgまでなら破れないナップサックを背負って美術館に華麗に侵入したとする．警報が鳴り出す前に，ナップサックが破れない範囲において，もっとも価値合計が大きくなるように美術品の数々を盗み出したい．どれを持ち帰るべきだろうか？

（想像するだけにしておくこと．佳き大学生として決して実行しないように要請する．）

美術品は全て１点ものだが，一応，市場価格があるものとする．つまり，各物体 i には，価値 V<sub>i</sub> と 重量 W<sub>i</sub> が定まっているものとする．いずれも正値とする．

**ヒューリスティックな探索**

賢い諸君がおそらく真っ先に考えるのは，美術品ごとに重量当たりの価値を求め，それをソーティングし，上位から順にナップサックを詰めていく方法であろう．

多くの場合，この考え方は現実的であるが，最適解を得られる保証はない．これは，ナップサック問題において，V<sub>i</sub> と W<sub>i</sub> がまちまちであることが原因である．（もし対象が美術品ではなく，砂金のように分割してもいいようなものであれば，この方法で最適解が得られる．） 




# 全解探索法


明らかに，各物体については，拾うか拾わないかの二択である．これをN回繰り返すので，解候補は 2<sup>N</sup> ある．解候補それぞれについて，

* ナックサックの重量制限内

であるかどうかを確認し，この条件に合致する中で最大価値を達成する組み合わせが解となる．

基数が2で小さいとはいえ，物体の数Nに対して解候補が2<sup>N</sup>になる．

ナップサック問題は問題のサイズNに対して，解の探索として指数時間かかるアルゴリズムしか発見されていない．このことから，ナップサック暗号というものが発明され，電子取引の最初期の基盤技術に利用されていたりしたことがある．

下記の実装例では，N=8において実行が2<sup>N</sup>すなわり256回程度になっていることに注意する．

In [ ]:
%%writefile Knapsack-bf_J.c
// Knapsack problem solution by Bluteforce
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h>
#include <stdlib.h> // atoi

typedef struct {int v; int w;} knapsack_t;

#define N 8

// Weight Limit
int wlimit = 0;

// (value, weight)
knapsack_t obj[N] =
{
	{ 15,  6},
	{100, 20},
	{ 90, 25},
	{ 60, 30},
	{ 40, 40},
	{ 15, 28},
	{ 16, 29},
	{  3, 10}
};

// 1...Pickup, 0...Discard
int stat[N]; 

// Maximum Value in the knapsack and tis weight
knapsack_t max = {0, 0};

// Show knapsack up to the object 0 ... currentobj
void showknapsack(int currentobj){
    int i;

	printf("List(value) at Object %d : ", currentobj);
	for (i = 0; i <= currentobj; i++) {
		printf("%3d ", stat[i] ? obj[i].v : 0);
	}
	for (i = currentobj + 1; i < N; i++) {
		printf("--- ");
	}
	printf(": val = %4d (weight = %4d)\n", max.v, max.w);
}

// Examine the object oid to pickup or not
// oid        ... Object ID to examine
// ks_current ... current status of the knapsack at this moment
void pickobject(int oid, knapsack_t ks_current){
	knapsack_t ks_next; // next status of the knapsack

    // End of the search
	if (oid >= N) {
        if (ks_current.w <= wlimit) {
            // New record (update the answer) ?
		    if (ks_current.v > max.v) {
                // Current knapsack is record-breaking
			    max.v = ks_current.v;
			    max.w = ks_current.w;
                printf("New record ! (%3d) ", max.v);
			    showknapsack(oid - 1);
	    	} else {
                printf("............ (%3d) ", ks_current.v);
			    showknapsack(oid - 1);              
            }
        } else {
            printf("...weight NG [%3d] ", ks_current.w);
		    showknapsack(oid - 1);              
        }
		return ;
	}

    //---------------------
    // Search : two choices
    //---------------------

    // ----
    // choice of Pickup (stat[oid] = 1)
    // ----

	// Let's think about PICKING UP the object oid from now.                     
	// stat[0] ... stat[i] with stat[i]=1
    {

		stat[oid] = 1; // Mark (Pick it up!)

		// Pick up object oid and proceed to the next object to examine
    	// When the object is taken, you have to update the knapsack status. 
		// [Recursive call]
		ks_next.v = ks_current.v + obj[oid].v;
		ks_next.w = ks_current.w + obj[oid].w;
		pickobject(oid + 1, ks_next);

		// On returning to here, we have done everything with stat[0] ... stat[i] with stat[i]=1
		// Okey, it's over, clean it up

		stat[oid] = 0; // Unmark
	}

	//----
	//　choice of Discard (stat[i] = 0)
	//----

	// Let's think about EXCLUDING the object oid from now.                     
	// stat[0] ... stat[i] with stat[i]=0
    {
    	// When the object is excluded, you don't need to update the knapsakc status. 

        // Nothing to mark

	    // Leave object oid and proceed to the next object to examine
	    pickobject(oid + 1, ks_current);

        // Nothing to unmark
    }

    return ;
}

// Main function
int main(int argc, char *argv[]){
	knapsack_t ks_init = {0, 0};

    // examine options
    if (argc != 2) {
        printf("Please set the weight limit (positive integer).\n");
        return -1;
    }
    wlimit = atoi(argv[1]);

    // Go
	pickobject(0, ks_init);
	
	printf("Result\n");
	printf("weight_limit = %d\n", wlimit);
	printf("maxv = %d (with weight %d)\n", max.v, max.w);
	return 0;
}


In [ ]:
!gcc -Wall -o Knapsack-bf_J Knapsack-bf_J.c

実行時には重み制限を引数として与える．ここでは112としてみる．

In [ ]:
!./Knapsack-bf_J 112

表示した行数を数えてみよう．wcコマンドを用いる(-lで行のみ数える)．最後の結果表示に3行用いているので，残りの256行が途中経過の表示に相当する．

In [ ]:
!./Knapsack-bf_J 112 | wc -l

# バックトラック法  
ナップサック問題では条件は「ナックサックの重量制限内」のみである．
そこで，これを解候補生成の際の条件にする．

全解探索のプログラム Knapsack-bf_J.c をもとにバックトラック法で書き下したプログラムが下記である．もとのプログラムをよく理解していれば，違いがどこに現れるか予想が付くであろう．まずは自分で確認してみること．









In [ ]:
%%writefile Knapsack-bt_J.c
// Knapsack problem solution by Backtrack
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h>
#include <stdlib.h> // atoi

typedef struct {int v; int w;} knapsack_t;

#define N 8

// Weight Limit
int wlimit = 0;

// (value, weight)
knapsack_t obj[N] =
{
	{ 15,  6},
	{100, 20},
	{ 90, 25},
	{ 60, 30},
	{ 40, 40},
	{ 15, 28},
	{ 16, 29},
	{  3, 10}
};

// 1...Pickup, 0...Discard
int stat[N]; 

// Maximum Value in the knapsack and tis weight
knapsack_t max = {0, 0};

// Show knapsack up to the object 0 ... currentobj
void showknapsack(int currentobj){
    int i;

	printf("List(value) at Object %d : ", currentobj);
	for (i = 0; i <= currentobj; i++) {
		printf("%3d ", stat[i] ? obj[i].v : 0);
	}
	for (i = currentobj + 1; i < N; i++) {
		printf("--- ");
	}
	printf(": val = %4d (weight = %4d)\n", max.v, max.w);
}

// Examine the object oid to pickup or not
// oid        ... Object ID to examine
// ks_current ... current status of the knapsack at this moment
void pickobject(int oid, knapsack_t ks_current){
	knapsack_t ks_next; // next status of the knapsack

    // End of the search
	if (oid >= N) {
        if (ks_current.w <= wlimit) {
            // New record (update the answer) ?
		    if (ks_current.v > max.v) {
                // Current knapsack is record-breaking
			    max.v = ks_current.v;
			    max.w = ks_current.w;
                printf("New record ! (%3d) ", max.v);
			    showknapsack(oid - 1);
	    	} else {
                printf("............ (%3d) ", ks_current.v);
			    showknapsack(oid - 1);              
            }
        } else {
            printf("...weight NG [%3d] ", ks_current.w);
		    showknapsack(oid - 1);              
        }
		return ;
	}

    //---------------------
    // Search : two choices
    //---------------------

    // ----
    // choice of Pickup (stat[oid] = 1)
    // ----

	// Let's think about PICKING UP the object oid from now.                     
	// stat[0] ... stat[i] with stat[i]=1
	if (ks_current.w + obj[oid].w <= wlimit) { // Weight limit condition

		stat[oid] = 1; // Mark (Pick it up!)

		// Pick up object oid and proceed to the next object to examine
    	// When the object is taken, you have to update the knapsack status. 
		// [Recursive call]
		ks_next.v = ks_current.v + obj[oid].v;
		ks_next.w = ks_current.w + obj[oid].w;
		pickobject(oid + 1, ks_next);

		// On returning to here, we have done everything with stat[0] ... stat[i] with stat[i]=1
		// Okey, it's over, clean it up

		stat[oid] = 0; // Unmark
	}

	//----
	//　choice of Discard (stat[i] = 0)
	//----

	// Let's think about EXCLUDING the object oid from now.                     
	// stat[0] ... stat[i] with stat[i]=0
    {
    	// When the object is excluded, you don't need to update the knapsakc status. 

        // Nothing to mark

	    // Leave object oid and proceed to the next object to examine
	    pickobject(oid + 1, ks_current);

        // Nothing to unmark
    }

    return ;
}

// Main function
int main(int argc, char *argv[]){
	knapsack_t ks_init = {0, 0};

    // examine options
    if (argc != 2) {
        printf("Please set the weight limit (positive integer).\n");
        return -1;
    }
    wlimit = atoi(argv[1]);

    // Go
	pickobject(0, ks_init);
	
	printf("Result\n");
	printf("weight_limit = %d\n", wlimit);
	printf("maxv = %d (with weight %d)\n", max.v, max.w);
	return 0;
}


二つのプログラムがとてもよく似ていることに気づいただろうか．
実は実質的に違いは１行のみである．
このことは，diffコマンドを使ってみればよくわかる．

(先頭行は単なるコメントなので違いとしてはノーカウント．）  


In [ ]:
!diff Knapsack-bf_J.c Knapsack-bt_J.c

(便利な世の中で https://www.diffchecker.com/ とかなども利用できる)
コンパイルしてエラーが無いことを確認．

In [ ]:
!gcc -Wall -o Knapsack-bt_J Knapsack-bt_J.c

同じ重み制限112を付けて実行してみよう．

In [ ]:
!./Knapsack-bt_J 112

行数も同じように数えてみる．

In [ ]:
!./Knapsack-bt_J 112 | wc -l

最後の3行を無視すると，実質174行である．つまり，256-174回がバックトラック法によって計算コストとして削減できたことになる．おおよそ半分弱である．この比率は，Nや品物の価値・重量，品物の登場順，ナップサックの重量制限が変われば変わることになる．

バックトラック法が全解探索法と同じだけコストがかかることはないが，ビッグオー表現としてこの削減量を定式化することは（残念ながら）難しい．

# 分岐限定法 (Branch-and-Bound)

この方法がこの節で学ぶ本題である．  

バックトラック法では，品物を拾うかどうかを考える時に，対象を拾っていく場合にナップサックが破れないことを解候補生成の条件にしていた．

それでは，対象を拾わないまま先に進む場合にも何かできることはないだろうか．

よく考えれば，ある物体iを拾わないということは，最大価値を達成できる見込みが薄れていくことを意味している．見込みが薄いだけならまだ探索を進める意味はあるが，この見込みが絶望に変わるなら，その先の探索をする必要は全くないことになる．

美術館に侵入して品物の吟味を始める前は，もしかすると全ての品物を持ち帰れるかもしれないという期待を胸に秘めていることであろう．その期待値 (achievable value, av) は，探索を進めて行くときに「拾わない」ことを選択する度に，その品物の価値分だけ減っていく．拾わないことにしたのだから当然である．
この期待値 av が，これまでの探索において達成できた（暫定）最大価値合計よりも小さくなってしまったら，その先の探索に意味はない．すなわち，これを，ある品物iを「拾わない」場合の探索打ち切り条件とできる．

下記の Knapsack-bb_J.c プログラムは Knapsack-bt_J.c から分岐限定法に合うように変更したものである．diffによって違いを示しているので，合わせて確認していこう．

In [ ]:
%%writefile Knapsack-bb_J.c
// Knapsack problem solution by Branch-and-Bound
// kameda[at]ccs.tsukuba.ac.jp, 2020.
#include <stdio.h>
#include <stdlib.h> // atoi

typedef struct {int v; int w;} knapsack_t;

#define N 8

// Weight Limit
int wlimit = 0;

// (value, weight)
knapsack_t obj[N] =
{
	{ 15,  6},
	{100, 20},
	{ 90, 25},
	{ 60, 30},
	{ 40, 40},
	{ 15, 28},
	{ 16, 29},
	{  3, 10}
};

// 1...Pickup, 0...Discard
int stat[N]; 

// Maximum Value in the knapsack and tis weight
knapsack_t max = {0, 0};

// Show knapsack up to the object 0 ... currentobj
void showknapsack(int currentobj){
    int i;

	printf("List(value) at Object %d : ", currentobj);
	for (i = 0; i <= currentobj; i++) {
		printf("%3d ", stat[i] ? obj[i].v : 0);
	}
	for (i = currentobj + 1; i < N; i++) {
		printf("--- ");
	}
	printf(": val = %4d (weight = %4d)\n", max.v, max.w);
}

// Examine the object oid to pickup or not
// oid        ... Object ID to examine
// ks_current ... current status of the knapsack at this moment
// av         ... achievable value (at best) ... new for branch and bound
void pickobject(int oid, knapsack_t ks_current, int av){
	knapsack_t ks_next; // next status of the knapsack
	int av_without_i; // achievable value at best without (i)th object

    // End of the search
	if (oid >= N) {
        if (ks_current.w <= wlimit) {
            // New record (update the answer) ?
		    if (ks_current.v > max.v) {
                // Current knapsack is record-breaking
			    max.v = ks_current.v;
			    max.w = ks_current.w;
                printf("New record ! (%3d) ", max.v);
			    showknapsack(oid - 1);
	    	} else {
                printf("............ (%3d) ", ks_current.v);
			    showknapsack(oid - 1);              
            }
        } else {
            printf("...weight NG [%3d] ", ks_current.w);
		    showknapsack(oid - 1);              
        }
		return ;
	}

    //---------------------
    // Search : two choices
    //---------------------

    // ----
    // choice of Pickup (stat[oid] = 1)
    // ----

	// Let's think about PICKING UP the object oid from now.                     
	// stat[0] ... stat[i] with stat[i]=1
	if (ks_current.w + obj[oid].w <= wlimit) { // Weight limit condition

		stat[oid] = 1; // Mark (Pick it up!)

		// Pick up object oid and proceed to the next object to examine
    	// When the object is taken, you have to update the knapsack status. 
		// [Recursive call]
		ks_next.v = ks_current.v + obj[oid].v;
		ks_next.w = ks_current.w + obj[oid].w;
		pickobject(oid + 1, ks_next, av); // av is same because you pick up this object.

		// On returning to here, we have done everything with stat[0] ... stat[i] with stat[i]=1
		// Okey, it's over, clean it up

		stat[oid] = 0; // Unmark
	}

	//----
	//　choice of Discard (stat[i] = 0)
	//----

	// Let's think about EXCLUDING the object oid from now.                     
	// stat[0] ... stat[i] with stat[i]=0
	av_without_i = av - obj[oid].v;	

	// [Branch and bound] here!
	// If 'achievable value' is smaller than 'max.v', there is no hope to
	// get better solution beyond here. So, you need to proceed only when
	//  av_without_i > max.v
	if (av_without_i > max.v) {


    	// When the object is excluded, you don't need to update the knapsakc status. 

        // Mark that no pick-up is made here : "stat[oid] = 0;" could be here.

	    // Leave object oid and proceed to the next object to examine
	  pickobject(oid + 1, ks_current, av_without_i); // less hope yet go

        // Unmark that recovery to the original state. "stat[oid] = 0;" could be here.
    }

    return ;
}

// Main function
int main(int argc, char *argv[]){
	knapsack_t ks_init = {0, 0};
	int totalvalue = 0;
	int i;

    // examine options
    if (argc != 2) {
        printf("Please set the weight limit (positive integer).\n");
        return -1;
    }
    wlimit = atoi(argv[1]);

    // Total Value
    for (i = 0; i < N; i++)
      totalvalue += obj[i].v;

    // Go
    pickobject(0, ks_init, totalvalue);
	
	printf("Result\n");
	printf("weight_limit = %d\n", wlimit);
	printf("maxv = %d (with weight %d)\n", max.v, max.w);
	return 0;
}


In [ ]:
!diff Knapsack-bt_J.c Knapsack-bb_J.c

コンパイルしてエラーがないことを確認．

In [ ]:
!gcc -Wall -o Knapsack-bb_J Knapsack-bb_J.c

同じ重み制限112を付けて実行してみよう．

In [ ]:
!./Knapsack-bb_J 112

明らかに解候補の生成が抑制されている．ナップサック問題の例では，バックトラック法と分岐限定法が，各品物iの「拾う」側と「拾わない」側のそれぞれで解の探索を抑制する形になるので，両方が揃うと途中であきらめる回数が増える．  
このことはすなわち生成される解候補が減ることを意味する．  
ここでの例では、解候補が2つにまで減ったことで，計算量の削減効果が劇的に表れている．


# 節末課題

1. 計算量についての考察  
Knsapsack-bf_Jプログラム, Knapsack-bt_Jプログラム, およびKnapsack-bb_Jプログラムそれぞれについて，時間計算量と空間計算量について考察せよ．特に，時間計算量については，幾つかのプログラムにおいては，計算量が同じになってしまう状況が考えられる．そのことについても言及せよ．


2. 計算量の実測  
ビッグオー表現でなく，簡単ではあるが各プログラムで計算量を実際に測定せよ．3つのプログラム Knapsack-bf_J, Knapsack-bt_J, Knapsack-bb_J において，pickup()関数の呼び出し回数を数えるプログラムを作成し，その数で3プログラムの計算コストを比較すること．ナップサックの重み制限を変更して，その変化を表にまとめること．重み制限を変えて10回以上実験すること．（つまり全体で30回以上試行すること)
結果を表にまとめ，考察もすること．







# 出典

筑波大学工学システム学類  
データ構造とアルゴリズム  
担当：亀田能成  
2021/06/16 初版